### Tools

In [ ]:
import pandas as pd
from langchain_core.messages import ToolMessage, HumanMessage, SystemMessage, AIMessage

def create_csv(filename, columns):
    """
    Creates a new CSV file with the specified filename and columns.
    
    Parameters:
    filename (str): The name of the CSV file to create (e.g., 'data.csv').
    columns (str or list of str, optional): A comma-separated string or a list of column names for the CSV. Defaults to None.
    
    Returns:
    None
    """
    # If columns is a string, split it into a list
    if isinstance(columns, str):
        columns = [col.strip() for col in columns.split(",")]
    
    # Initialize an empty DataFrame with specified columns if provided
    df = pd.DataFrame(columns=columns)
    # Save the empty DataFrame to a CSV file
    df.to_csv(filename, index=False)
    return {'output': (f"CSV file '{filename}' created successfully.")}

import os
import glob

def list_csv_files_with_columns():
    """
    Lists all CSV files along with their column names in the specified folder.

    Returns:
    list: A list of tuples, each containing a CSV file name and its column names.
    """
    try:
        folder_path=''
        # Use glob to find all CSV files in the specified folder
        csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

        csv_info = []

        for file in csv_files:
            try:
                # Read the CSV file to extract column names
                df = pd.read_csv(file)
                columns = df.columns.tolist()
                
                # Add filename and columns as a tuple to the list
                csv_info.append(
                    {
                    'filename':os.path.basename(file), 
                    'columns': columns
                }
                )
            except pd.errors.EmptyDataError:
                # Handle empty files specifically
                csv_info.append({'filename': os.path.basename(file), 
                                 'Error': 'File is empty'}
                )
            except pd.errors.ParserError:
                # Handle malformed files specifically
                csv_info.append({'filename': os.path.basename(file), 
                                 'Error': 'Malformed file'}
                )
            except Exception as e:
                # Handle any other exceptions
                csv_info.append({'filename': os.path.basename(file), 
                                 'Error': str(e)}
                )

        return {'output': csv_info}

    except Exception as e:
        return {'error': f"Error: {e}"}


def read_csv_file(filename):
    """
    Reads a CSV file and returns its contents as a DataFrame.
    
    Parameters:
    filename (str): The name of the CSV file to read (e.g., 'data.csv').
    
    Returns:
    pd.DataFrame: A DataFrame containing the CSV file's data.
    """
    try:
        # Read the CSV file
        df = pd.read_csv(filename)
        return {'output': df}
    except FileNotFoundError:
        return {'error' : (f"Error: The file '{filename}' was not found.")}
    except pd.errors.EmptyDataError:
        return {'error' :(f"Error: The file '{filename}' is empty.")}
    except pd.errors.ParserError:
        return {'error' :(f"Error: The file '{filename}' contains parsing errors.")}

import json
import pandas as pd

def add_row_to_csv(filename, data):
    """
    Adds a new row to an existing CSV file using data provided in JSON string format.
    Validates that the columns in the data match the CSV file columns.

    Parameters:
    filename (str): The name of the CSV file to modify.
    data (str): A JSON string representing the row data to add (keys should match column names).

    Returns:
    dict: Output or error message.
    """
    try:
        # Parse the JSON string to a dictionary
        data_dict = json.loads(data)

        # Read the existing CSV file into a DataFrame
        df = pd.read_csv(filename)

        # Get CSV columns and data dictionary keys
        csv_columns = set(df.columns)
        data_keys = set(data_dict.keys())

        # Find unmatched columns and keys
        unmatched_columns = data_keys - csv_columns
        unmatched_keys = csv_columns - data_keys

        # Check for mismatches in keys
        if unmatched_columns or unmatched_keys:
            error_message = "Data keys and CSV columns do not match."
            if unmatched_columns:
                error_message += f" Keys not in CSV columns: {', '.join(unmatched_columns)}."
            if unmatched_keys:
                error_message += f" CSV columns not in data: {', '.join(unmatched_keys)}."
            return {'error': error_message}

        # Convert the data dictionary to a DataFrame with a single row
        new_row = pd.DataFrame([data_dict])

        # Concatenate the new row to the existing DataFrame
        df = pd.concat([df, new_row], ignore_index=True)

        # Save the updated DataFrame back to the CSV file
        df.to_csv(filename, index=False)
        return {'output': f"Row added to '{filename}' successfully."}

    except json.JSONDecodeError:
        return {'error': "Error: Data is not valid JSON."}
    except FileNotFoundError:
        return {'error': f"Error: The file '{filename}' was not found."}
    except pd.errors.EmptyDataError:
        return {'error': f"Error: The file '{filename}' is empty."}
    except ValueError as e:
        return {'error': f"Error: {e}"}



def edit_dataframe_content(filename, index, column, new_value):
    """
    Edits the content of a DataFrame loaded from a CSV file at a specified index and column with new data.

    Parameters:
    filename (str): The name of the CSV file to read and modify.
    index (int or str): The index location to be edited.
    column (str): The column name where the data needs to be updated.
    new_value: The new value to be inserted at the specified index and column.

    Returns:
    dict: A message indicating success or an error encountered during the process.
    """
    try:
        # Load the CSV file into a DataFrame
        df = pd.read_csv(filename)

        # Check if the index and column are valid
        if column not in df.columns:
            return {'error': f"Column '{column}' not found in the CSV."}

        if index not in df.index:
            return {'error': f"Index '{index}' not found in the CSV."}

        # Update the DataFrame with the new value
        df.at[index, column] = new_value

        # Save the updated DataFrame back to the CSV
        df.to_csv(filename, index=False)

        return {'output': f"Data updated successfully at index '{index}' and column '{column}'."}

    except FileNotFoundError:
        return {'error': f"Error: The file '{filename}' was not found."}
    except ValueError as e:
        return {'error': f"Error: {e}"}
    except Exception as e:
        return {'error': f"Error: {e}"}


def delete_row_from_csv(filename, index):
    """
    Deletes a row from a CSV file based on the specified index.

    Parameters:
    filename (str): The name of the CSV file to modify.
    index (int): The index of the row to be deleted.

    Returns:
    dict: A message indicating success or an error encountered during the process.
    """
    try:
        # Load the CSV file into a DataFrame
        df = pd.read_csv(filename)

        # Check if the specified index exists
        if index not in df.index:
            return {'error': f"Index '{index}' not found in the CSV."}

        # Drop the row from the DataFrame
        df = df.drop(index)

        # Save the updated DataFrame back to the CSV
        df.to_csv(filename, index=False)

        return {'output': f"Row at index '{index}' deleted successfully."}

    except FileNotFoundError:
        return {'error': f"Error: The file '{filename}' was not found."}
    except ValueError as e:
        return {'error': f"Error: {e}"}
    except Exception as e:
        return {'error': f"Error: {e}"}







In [ ]:
tools = [
    create_csv,
    add_row_to_csv,
    read_csv_file,
    list_csv_files_with_columns,
    edit_dataframe_content,
    delete_row_from_csv
]

def _make_tool_msg(msg, tool_id):
    if 'error' in msg:
        status='error'
        content = msg['error']
    else:
        status='success'
        content = str(msg['output'])
        
    return ToolMessage(
            content=content,
            tool_call_id=tool_id,
            status=status
    )
    
def call_tools(msg: AIMessage):
    """Simple sequential tool calling helper."""
    try:
        tool_results = []
        tool_map = {tool.__name__: tool for tool in tools}
        tool_calls = msg.tool_calls.copy()
        for tool_call in tool_calls:
            tool_results.append(
                _make_tool_msg(
                    tool_map[tool_call["name"]](**tool_call["args"]),
                    tool_id=tool_call['id']
                )
            )
        return tool_results
    except Exception as e:
        tool_results.append(
            ToolMessage(
                content=f"Calling tool with arguments:\n\n{tool_call["args"]}\n\nraised the following error:\n\n{type(e)}: {e}",
                tool_call_id=tool_call['id'],
                status='error'
        ))
        return tool_results


### Model

#### openai

In [ ]:
import os

openai_key = os.environ['OPENAI_API_KEY']

from langchain_openai import ChatOpenAI
from langchain_core.messages import ToolMessage, HumanMessage, SystemMessage, AIMessage

from langchain_core.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

llm_openai  = ChatOpenAI(
    api_key=openai_key,
    model='gpt-4o-mini'
)

# human_msg = HumanMessage()
system_msg = SystemMessage(content="Your an personal assistant who help in managing the csv")

In [ ]:
# llm.invoke("hi")

#### grok

In [ ]:
from langchain_groq import ChatGroq
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI

groq_key = os.environ['GROQ_API_KEY']

llm_grok = ChatGroq(
    model="llama-3.2-90b-vision-preview",
    api_key=groq_key,
    temperature=0.7,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)


#### bedrock

In [ ]:
from langchain_aws import ChatBedrock, ChatBedrockConverse

llm_bed = ChatBedrock(
    model="us.meta.llama3-2-90b-instruct-v1:0",
    aws_access_key_id = '',
    aws_secret_access_key = "" ,
    region_name = '',
    provider=''
    # other params...
)

### Agent

In [ ]:
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage

from langgraph.graph import START, StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [ ]:
class Agent:
    def __init__(self, llm, tools):
        self.llm_with_tool = llm.bind_tools(tools)
        graph = StateGraph(AgentState)
        graph.add_node('llm', self.call_llm)
        graph.add_node('tools', self.tools)

        graph.add_conditional_edges(
            'llm',
            self.tool_check,
            {True: 'tools', False: END}
        )
        graph.add_edge(
            'tools',
            'llm'
        )
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        # self.graph = graph.compile(checkpointer=MemorySaver())
        
            

    def call_llm(self, state):
        return {'messages': [self.llm_with_tool.invoke(state['messages'])]}

    def tools(self, state):
        return {'messages': call_tools(state['messages'][-1])}

    def tool_check(self, state):
        return len(state['messages'][-1].tool_calls) > 0
    

In [ ]:
graph = Agent(llm_bed, tools)

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
system_msg = SystemMessage(
    content=(
        '''
**Instructions**: Act as a personal assistant and CSV file manager while maintaining transparent communication for tool interactions.

**Details**:

1. **CSV File Management**:
   - Manage existing CSV files by opening, analyzing, or editing data without redundancy.
   - Before creating a new CSV file, ensure one doesn't already exist to prevent duplicates.
   - Perform necessary data operations like filtering or updating with efficiency and accuracy.

2. **Tool Interaction Transparency**:
   - If a tool call is made, explain the specific reason for the action. Clearly outline why it was necessary to use the tool and how it benefits the task at hand.

**Formatting Requirements**:
- Deliver concise and clear responses for personal assistant queries.
- Present CSV-related operations in structured text or tables.
- Provide explanations for tool interactions with justifications for their necessity.

Ensure a user-friendly experience while maintaining transparency and efficiency in both personal assistant and CSV data management tasks.
'''
    )
)


In [ ]:
input_msg = input()

messages = [
    system_msg,
    HumanMessage(content=input_msg)
                 
]
for event in graph.graph.stream({"messages": messages}, stream_mode="values"):
    event["messages"][-1].pretty_print()
